In [4]:
import duckdb 
query = """
	WITH recursive mvt as materialized(
SELECT
	DISTINCT mod,
	date_eff,
	com_av,
	libelle_av,
	com_ap,
	libelle_ap
FROM
	read_csv('https://www.insee.fr/fr/statistiques/fichier/7766585/v_mvt_commune_2024.csv')
WHERE
	typecom_av = 'COM'
	AND typecom_ap = 'COM'
	AND NOT (mod <> 21
		AND com_av = com_ap)
	AND mod NOT in (20, 30)
	),
	start_com as (
SELECT
	*
FROM
	(
	SELECT
		*,
		max(date_eff) OVER(PARTITION BY com_av) as firstdate
	FROM
		mvt)
WHERE
	date_eff = firstdate),
	com_cte(com_av,
libelle_av,
date_eff,
com_av2,
com_ap,
libelle_ap,
count_) as (
SELECT
	start_com.com_av as com_av,
	start_com.libelle_av as libelle_av,
	start_com.date_eff as date_eff,
	start_com.com_av as com_av2 ,
	start_com.com_ap as com_ap,
	start_com.libelle_ap as libelle_ap,
	1 as count_
FROM
	start_com
UNION ALL
SELECT
	com_cte.com_av as com_av,
	com_cte.libelle_av as libelle_av,
	mt.date_eff as date_eff,
	mt.com_av as com_av2 ,
	mt.com_ap as com_ap,
	mt.libelle_ap as libelle_ap,
	com_cte.count_ + 1 as count_
FROM
	com_cte
JOIN mvt as mt on
	com_cte.com_ap = mt.com_av
	and com_cte.date_eff <= mt.date_eff
WHERE
	not (mt.mod = 21
		AND com_av2 <> mt.com_ap)
	AND count_ < 10
	),
	final_result as (
SELECT
	distinct com_av,
	libelle_av,
	com_ap,
	libelle_ap
FROM
	(
	SELECT
		*,
		max(count_) OVER(PARTITION BY com_av) as maxcount
	FROM
		com_cte)
WHERE
	count_ = maxcount)
	SELECT
	*
FROM
	final_result

"""
duckdb.sql(query).write_csv("passage_communes_1943.csv")
duckdb.sql(query).show()

┌─────────┬─────────────────────────┬─────────┬───────────────────────────┐
│ com_av  │       libelle_av        │ com_ap  │        libelle_ap         │
│ varchar │         varchar         │ varchar │          varchar          │
├─────────┼─────────────────────────┼─────────┼───────────────────────────┤
│ 08294   │ La Moncelle             │ 08053   │ Bazeilles                 │
│ 14139   │ Carville                │ 14061   │ Souleuvre en Bocage       │
│ 14671   │ Sept-Frères             │ 14658   │ Noues de Sienne           │
│ 20021   │ Argiusta-Moriccio       │ 2A021   │ Argiusta-Moriccio         │
│ 20276   │ Serra-di-Ferro          │ 2A276   │ Serra-di-Ferro            │
│ 20358   │ Zévaco                  │ 2A358   │ Zévaco                    │
│ 25399   │ Montfort                │ 25460   │ Le Val                    │
│ 27236   │ Faverolles-les-Mares    │ 27106   │ Bournainville-Faverolles  │
│ 28340   │ Saint-Hilaire-sur-Yerre │ 28103   │ Cloyes-les-Trois-Rivières │
│ 35301   │ 